In [80]:
#imports
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import shutil
import random
from pathlib import Path
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [81]:
#Define paths
BASE_PATH = Path('D:/IIMS/sem 3/Computer Vision And Natural Language Processing/datasets')
OUTPUT_PATH = BASE_PATH / 'combined_dataset'

NP_IMAGES = [
    BASE_PATH / 'number plates/normal_imgs',
    #BASE_PATH / 'number plates/wide_imgs',
    BASE_PATH / 'number plates/zoomed_imgs'
]
NP_LABELS = [
    BASE_PATH / 'number plates/normal_number_plate_labels',
    #BASE_PATH / 'number plates/wide_number_plate_labels',
    BASE_PATH / 'number plates/zoomed_number_plate_labels'
]

#Vehicle Dataset
VEHICLE_IMAGES = [
    #BASE_PATH / 'vehicles/wide_imgs',
    BASE_PATH / 'vehicles/all_normal_vehicle_dataset_with_labels/car/images',
    BASE_PATH / 'vehicles/all_normal_vehicle_dataset_with_labels/bikes/images',
    BASE_PATH / 'vehicles/all_normal_vehicle_dataset_with_labels/scooter/images'
]
VEHICLE_LABELS = [
    #BASE_PATH / 'vehicles/wide_vehicle_detection_labels',
    BASE_PATH / 'vehicles/all_normal_vehicle_dataset_with_labels/car/labels',
    BASE_PATH / 'vehicles/all_normal_vehicle_dataset_with_labels/bikes/labels',
    BASE_PATH / 'vehicles/all_normal_vehicle_dataset_with_labels/scooter/labels'
]

CLASS_NAMES = ['number_plate', '4_wheeler', '2_wheeler']
CLASS_IDS = [0, 1, 2]

In [82]:
#combine dataset
def create_dataset_structure():
    for folder in ['images/train', 'images/val', 'images/test', 'labels/train', 'labels/val', 'labels/test']:
        (OUTPUT_PATH / folder).mkdir(parents=True, exist_ok=True)

In [83]:
def merge_and_split_datasets(train_split=0.7, val_split=0.2):
    class_images = {0: [], 1: [], 2: []}  # 0: number_plate, 1: 4_wheeler, 2: 2_wheeler
    class_labels = {0: [], 1: [], 2: []}
    id_mapping = {0: 0, 1: 1, 2: 2, 11: 2}  # Map scooter (11) to 2_wheeler (2)

    # Process number plate dataset
    print("Processing number plate dataset...")
    for img_dir, lbl_dir in zip(NP_IMAGES, NP_LABELS):
        print(f"Checking directory: {img_dir}")
        img_count = 0
        for img in img_dir.glob('*.jpg'):
            img_count += 1
            label_file = lbl_dir / f"{img.stem}.txt"
            if label_file.exists():
                with open(label_file, 'r') as f:
                    lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if parts:
                        original_class_id = int(parts[0])
                        class_id = id_mapping.get(original_class_id, -1)
                        if class_id in [0]:  # number_plate
                            class_images[class_id].append(img)
                            class_labels[class_id].append(label_file)
                            break  # Assuming one dominant class per image
                        elif class_id == -1:
                            print(f"Warning: Unknown class ID {original_class_id} in {label_file}")
        print(f"Found {img_count} images in {img_dir}")

    # Process vehicle dataset
    print("Processing vehicle dataset...")
    for img_dir, lbl_dir in zip(VEHICLE_IMAGES, VEHICLE_LABELS):
        print(f"Checking directory: {img_dir}")
        img_count = 0
        for img in img_dir.glob('*.jpg'):
            img_count += 1
            label_file = lbl_dir / f"{img.stem}.txt"
            if label_file.exists():
                with open(label_file, 'r') as f:
                    lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if parts:
                        original_class_id = int(parts[0])
                        class_id = id_mapping.get(original_class_id, -1)
                        if class_id in [1, 2]:  # 4_wheeler or 2_wheeler
                            class_images[class_id].append(img)
                            class_labels[class_id].append(label_file)
                            break  # Assuming one dominant class per image
                        elif class_id == -1:
                            print(f"Warning: Unknown class ID {original_class_id} in {label_file}")
        print(f"Found {img_count} images in {img_dir}")

    # Print the number of images per class
    print("\nImages per class before splitting:")
    print(f"Number Plate (0): {len(class_images[0])}")
    print(f"4-Wheeler (1): {len(class_images[1])}")
    print(f"2-Wheeler (2): {len(class_images[2])}")

    # Split each class into train/val/test
    train_images, train_labels = [], []
    val_images, val_labels = [], []
    test_images, test_labels = [], []

    for class_id in class_images.keys():
        images = class_images[class_id]
        labels = class_labels[class_id]

        if not images:  # Skip if there are no images for this class
            print(f"No images found for class {class_id}, skipping split.")
            continue

        # Shuffle the images and labels for this class
        combined = list(zip(images, labels))
        random.shuffle(combined)
        images, labels = zip(*combined)

        # Calculate split indices
        total = len(images)
        train_idx = int(total * train_split)  # 70%
        val_idx = int(total * (train_split + val_split))  # 70% + 20%

        # Split the data
        train_images.extend(images[:train_idx])
        train_labels.extend(labels[:train_idx])
        val_images.extend(images[train_idx:val_idx])
        val_labels.extend(labels[train_idx:val_idx])
        test_images.extend(images[val_idx:])
        test_labels.extend(labels[val_idx:])

    # Shuffle the final splits to mix classes
    train_combined = list(zip(train_images, train_labels))
    random.shuffle(train_combined)
    train_images, train_labels = zip(*train_combined) if train_combined else ([], [])

    val_combined = list(zip(val_images, val_labels))
    random.shuffle(val_combined)
    val_images, val_labels = zip(*val_combined) if val_combined else ([], [])

    test_combined = list(zip(test_images, test_labels))
    random.shuffle(test_combined)
    test_images, test_labels = zip(*test_combined) if test_combined else ([], [])

    print(f"\nTrain split: {len(train_images)} images")
    print(f"Validation split: {len(val_images)} images")
    print(f"Test split: {len(test_images)} images")

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

In [84]:
#Copy files and update labels
def copy_files(images, labels, split_type):
    class_mapping = {'number_plate': 0, '4_wheeler': 1, '2_wheeler': 2}
    
    for img, lbl in zip(images, labels):
        shutil.copy(img, OUTPUT_PATH / f'images/{split_type}/{img.name}')
        
        with open(lbl, 'r') as f:
            lines = f.readlines()
        
        with open(OUTPUT_PATH / f'labels/{split_type}/{lbl.name}', 'w') as f:
            for line in lines:
                parts = line.strip().split()
                if parts:
                    
                    # Map original class IDs to new ones (assuming original dataset has different IDs)
                    class_name = next((k for k, v in class_mapping.items() if v == int(parts[0])), None)
                    if class_name:
                        parts[0] = str(class_mapping[class_name])
                    else:
                        # If the class ID isn't in mapping, keep it as is (for flexibility)
                        pass
                    f.write(' '.join(parts) + '\n')

In [85]:
#creating data.yaml
def create_data_yaml():
    yaml_content = f"""
path: {OUTPUT_PATH}
train: images/train
val: images/val
test: images/test
nc: 4
names: ['number_plate', '4_wheeler', '2_wheeler', '']
    """
    with open(OUTPUT_PATH / 'data.yaml', 'w') as f:
        f.write(yaml_content)

In [86]:
#Main dataset preparation function
def prepare_dataset():
    create_dataset_structure()
    train_img, train_lbl, val_img, val_lbl, test_img, test_lbl = merge_and_split_datasets()
    copy_files(train_img, train_lbl, 'train')
    copy_files(val_img, val_lbl, 'val')
    copy_files(test_img, test_lbl, 'test')
    create_data_yaml()
    print("Dataset preparation completed!")

In [87]:
def train_and_export_models(test_images, test_labels):
    device = 0 if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device} {'(GPU)' if torch.cuda.is_available() else '(CPU)'}")
    
    model = YOLO('yolov8n.pt')
    print("\nModel Summary:")
    print(model)
    
    # Train the model
    results = model.train(
        data=str(OUTPUT_PATH / 'data.yaml'),
        epochs=50,  # Training for 5 epochs
        batch=16,
        imgsz=640,
        device=device,
        patience=20,
        verbose=True
    )

    # Define epochs
    epochs = list(range(1, 51))  # [1, 2, 3, 4, 5], length 5

    import pandas as pd
    try:
        # The training results are saved in runs/detect/train/results.csv (or similar)
        results_csv = 'runs/detect/train/results.csv'  # Adjust path if needed
        df = pd.read_csv(results_csv)
        # The column for mAP@50 is typically 'metrics/mAP50(B)'
        accuracy = df['metrics/mAP50(B)'].tolist()  # Extract mAP@50 for each epoch
        print(f"Extracted mAP@50 per epoch: {accuracy}")
    except Exception as e:
        print(f"Could not extract per-epoch metrics: {e}")
        # Fallback: Use the final mAP@50 and repeat it (not ideal, but ensures compatibility)
        final_map50 = results.results_dict.get('metrics/mAP50(B)', 0)
        accuracy = [final_map50] * 50  # Repeat the final mAP@50 for each epoch

    # Ensure accuracy matches the length of epochs
    if len(accuracy) != len(epochs):
        print(f"Warning: Length of accuracy ({len(accuracy)}) does not match epochs ({len(epochs)}). Truncating/padding accuracy.")
        if len(accuracy) > len(epochs):
            accuracy = accuracy[:len(epochs)]  # Truncate to match epochs
        else:
            accuracy.extend([accuracy[-1]] * (len(epochs) - len(accuracy)))  # Pad with the last value

    # Save and export the model
    model.save('npvd.pt')
    model.export(format='onnx', imgsz=640)
    print("Model exported to npvd.onnx")
    
    return model, accuracy, epochs, test_images, test_labels

In [88]:
#Training and exporting function
def train_and_export_model(test_images, test_labels):
    device = 0 if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device} {'(GPU)' if torch.cuda.is_available() else '(CPU)'}")
    
    model = YOLO('yolov8n.pt')
    print("\nModel Summary:")
    print(model)
    
    results = model.train(
        data=str(OUTPUT_PATH / 'data.yaml'),
        epochs=50,
        batch=16,
        imgsz=640,
        device=device,
        patience=20,
        verbose=True
    )
    
    #Extract metrics per epoch
    epochs = range(1, 51)
    accuracy = [results.results_dict.get('metrics/mAP50(B)', 0)] * 50
    
    model.save('npvd.pt')
    model.export(format='onnx', imgsz=640)
    print("Model exported to npvd.onnx")
    
    return model, accuracy, epochs, test_images, test_labels

In [89]:
#model evalutation function
def evaluate_model(model, test_images, test_labels, test_videos=None, test_images_to_detect=None):
    y_true, y_pred = [], []
    for img, lbl in zip(test_images, test_labels):
        with open(lbl, 'r') as f:
            gt = [int(line.split()[0]) for line in f.readlines() if line.strip()]
        results = model.predict(img, save=False)
        pred = results[0].boxes.cls.cpu().numpy().astype(int) if results[0].boxes else []
        
        for cid in CLASS_IDS:
            y_true.append(1 if cid in gt else 0)
            y_pred.append(1 if cid in pred else 0)
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
    plt.title('Confusion Matrix')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    accuracy = sum(1 for t, p in zip(y_true, y_pred) if t == p) / len(y_true)
    print(f"\nTest Accuracy: {accuracy:.4f}")
    return accuracy

In [90]:
#plotting function for metrics
def plot_metrics(epochs, accuracy):
    plt.figure(figsize=(8, 4))
    plt.plot(epochs, accuracy, label='mAP@50')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('mAP@50')
    plt.legend()
    plt.grid(True)
    plt.show()

In [91]:
#detection functions
def detect_from_image(model, image_path, output_dir="output"):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    results = model.predict(image_path, save=False)
    annotated_img = results[0].plot()
    output_file = output_path / f"detected_{Path(image_path).name}"
    cv2.imwrite(str(output_file), annotated_img)
    print(f"Image detection saved to {output_file}")

def detect_from_video(model, video_path, output_dir="output"):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    output_file = output_path / f"detected_{Path(video_path).stem}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_file), fourcc, fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model.predict(frame, save=False)
        annotated_frame = results[0].plot()
        out.write(annotated_frame)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Video detection saved to {output_file}")

In [92]:
#main function
if __name__ == "__main__":
    
    #Prepare dataset
    prepare_dataset()
    
    torch.cuda.empty_cache()
    
    #Load test set
    _, _, _, _, test_images, test_labels = merge_and_split_datasets()
    
    #Train and export model
    trained_model, accuracy, epochs, _, _ = train_and_export_models(test_images, test_labels)
    
    #Test on all test images
    print("\nTesting on all test images:")
    for img in test_images:
        detect_from_image(trained_model, img)
    
    #Example inference on specific test files
    test_image = "D:/IIMS/sem 3/Computer Vision And Natural Language Processing/datasets/test/test.jpeg"
    test_video = "D:/IIMS/sem 3/Computer Vision And Natural Language Processing/datasets/test/test.mp4"
    detect_from_image(trained_model, test_image)
    detect_from_video(trained_model, test_video)
    
    test_accuracy = evaluate_model(trained_model, test_images, test_labels, test_video)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    plot_metrics(epochs, accuracy)

Processing number plate dataset...
Checking directory: D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\number plates\normal_imgs
Found 806 images in D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\number plates\normal_imgs
Checking directory: D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\number plates\zoomed_imgs


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000187BBAECCA0>
Traceback (most recent call last):
  File "c:\Users\Asure\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "c:\Users\Asure\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1436, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Found 2503 images in D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\number plates\zoomed_imgs
Processing vehicle dataset...
Checking directory: D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\vehicles\all_normal_vehicle_dataset_with_labels\car\images
Found 0 images in D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\vehicles\all_normal_vehicle_dataset_with_labels\car\images
Checking directory: D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\vehicles\all_normal_vehicle_dataset_with_labels\bikes\images
Found 54 images in D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\vehicles\all_normal_vehicle_dataset_with_labels\bikes\images
Checking directory: D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\vehicles\all_normal_vehicle_dataset_with_labels\scooter\images
Found 36 images in D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\vehicle

train: Scanning D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\combined_dataset\labels\train.cache... 3399 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3399/3399 [00:00<?, ?it/s]
val: Scanning D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\combined_dataset\labels\val... 3394 images, 0 backgrounds, 14 corrupt: 100%|██████████| 3394/3394 [00:02<00:00, 1555.52it/s]

val: WARNING  D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\combined_dataset\images\val\scooter_114.jpg: ignoring corrupt image/label: Label class 11 exceeds dataset class count 4. Possible class labels are 0-3
val: WARNING  D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\combined_dataset\images\val\scooter_151.jpg: ignoring corrupt image/label: Label class 11 exceeds dataset class count 4. Possible class labels are 0-3
val: WARNING  D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\combined_dataset\images\val\scooter_176.jpg: ignoring corrupt image/label: Label class 11 exceeds dataset class count 4. Possible class labels are 0-3
val: WARNING  D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\combined_dataset\images\val\scooter_206.jpg: ignoring corrupt image/label: Label class 11 exceeds dataset class count 4. Possible class labels are 0-3
val: WARNING  D:\IIMS\sem 3\Computer Vision And Natural 

...

                   all       3380      21005      0.692      0.675      0.658      0.568



50 epochs completed in 1.102 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.2MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.91  Python-3.10.0 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 106/106 [00:35<00:00,  3.02it/s]


                   all       3380      21005      0.692      0.675      0.658      0.568
          number_plate       3229      20051      0.989      0.984      0.994      0.878
             4_wheeler        455        463      0.833      0.724      0.682      0.606
             2_wheeler        463        483      0.947      0.994      0.956      0.788
                                8          8          0          0          0          0
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\train3
Could not extract per-epoch metrics: [Errno 2] No such file or directory: 'runs/detect/train/results.csv'
Ultralytics 8.3.91  Python-3.10.0 torch-2.1.0+cu118 CPU (13th Gen Intel Core(TM) i7-13700HX)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train3\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (6.0 MB)

ONNX: starting 

<Figure size 800x400 with 1 Axes>

In [96]:
test = "D:/IIMS/sem 3/Computer Vision And Natural Language Processing/datasets/test/two-wheeler.jpg"
detect_from_image(trained_model, test)


image 1/1 D:\IIMS\sem 3\Computer Vision And Natural Language Processing\datasets\test\two-wheeler.jpg: 448x640 1 4_wheeler, 7 2_wheelers, 151.5ms
Speed: 2.9ms preprocess, 151.5ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)
Image detection saved to output\detected_two-wheeler.jpg


In [ ]:
test_video = "D:/IIMS/sem 3/Computer Vision And Natural Language Processing/datasets/test/test.mp4"
detect_from_image(trained_model, test_video)